In [1]:
function T_local(q,c)
    return -1/log(1-(q/(1+sqrt(c-1))))
end
    

T_local (generic function with 1 method)

In [2]:
using Revise, Genie, StatsBase, Statistics, JLD2, DCAUtils, PyPlot

using Graphs, Random

function generate_random_regular_graph(L, d)
    @assert d < n "Degree must be less than the number of nodes"
    @assert iseven(n * d) "n*d must be even for a valid d-regular graph"

    edges = Tuple{Int, Int}[]
    stubs = repeat(1:n, d)
    shuffle!(stubs)

    while !isempty(stubs)
        u, v = pop!(stubs), pop!(stubs)
        if u != v && !((u, v) in edges || (v, u) in edges)
            push!(edges, (u, v))
        else
            push!(stubs, u)
            push!(stubs, v)
            shuffle!(stubs)
        end
    end

    G = SimpleGraph(n)
    for (u, v) in edges
        add_edge!(G, u, v)
    end
    return G
end





L = 50; c = 5; temp = 0.30; q = 4;

# Example usage:
n, d = 50, 5  # 100 nodes, degree 3
coupl = 1.0  # Antiferromagnetic interaction
G = generate_random_regular_graph(n, d)
connections = [neighbors(G,i) for i in 1:50];
J = zeros(q,L,q,L);
h = zeros(q,L)
for i in 1:L
    for j in connections[i]
        for a in 1:q
            J[a,i,a,j] = -1
        end
    end 
end

N_steps = 10^5; N_chains = 5000; start_msa = Int8.(rand(1:q,L, N_chains));
mean_dist = [];mean_intra_dist = [];mean_t2_dist = []; steps = []; steps_t2 = [];

@time res_eq = run_potts(start_msa, 
        h, 
        J, 
        temp = temp,
        each_step = 1000,
        N_steps = N_steps, 
        q = q);

for n in 1:length(res_eq.steps)  
    push!(mean_dist, mean(ham_dist(start_msa, res_eq.step_msa[n])))
    push!(mean_intra_dist, pairwise_ham_dist(res_eq.step_msa[n], n_seq = 500))
    push!(steps, res_eq.steps[n])
    if length(findall(x -> x == ((res_eq.steps[n]-1)/2)+1, res_eq.steps)) == 1
        push!(steps_t2, res_eq.steps[n])
        ind = findall(x -> x == ((res_eq.steps[n]-1)/2)+1, res_eq.steps)[1]
        push!(mean_t2_dist, mean(Genie.ham_dist(res_eq.step_msa[n], res_eq.step_msa[ind]))) 
    end
end

close("all"); 
    plt.plot(steps ./ L, mean_dist ./ L)
 plt.xscale("log");  plt.xlabel("Sweeps"); plt.ylabel("Hamming_from_start"
    ); savefig("../ciao_dist.png")

close("all"); 
    plt.plot(steps ./ L, mean_intra_dist ./ L)
    plt.scatter(steps_t2 ./ L, mean_t2_dist ./ L)
plt.xscale("log");  plt.xlabel("Sweeps"); plt.ylabel("Pairwise Hamming"
    ); savefig("../ciao_intra_dist.png")



rand_msa = Int8.(rand(1:q,L, N_chains));
f1_0,f2_0 = compute_weighted_frequencies(rand_msa, q+1,0.); c_0 = reshape(f2_0 - f1_0*f1_0', q, L, q, L
    );f1_0 = reshape(f1_0,q,L); f2_0 = reshape(f2_0,q,L,q,L);

f1,f2 = compute_weighted_frequencies(res_eq.step_msa[end], q+1,0.); c= reshape(f2 - f1*f1', q, L, q, L);
       f1 = reshape(f1,q,L); f2 = reshape(f2,q,L,q,L);

indices = partialsortperm(J[:], 1:1000, rev=true);

close("all"); plt.scatter(c_0[indices], c[indices]);plt.plot(
    [minimum(c_0[indices]), maximum(c_0[indices])],[minimum(c_0[indices]), maximum(c_0[indices])]
    ); savefig("../trial.png")

    
Genie.print_fasta_to_file_rna(res_eq.step_msa[end]',"../eq_sample_potts_t0.3_L50_d5.fa","coloring")
@save "../eq_sample_potts_t0.3_L50_d5.jld2" h J 


 63.354800 seconds (98.90 M allocations: 2.459 GiB, 0.86% gc time, 1.98% compilation time: 29% of which was recompilation)
θ = 0.0 threshold = 0.0
M = 5000 N = 50 Meff = 5000
θ = 0.0 threshold = 0.0
M = 5000 N = 50 Meff = 5000


LoadError: UndefVarError: `matrix2fasta` not defined

In [ ]:
using Revise, PhyloTools, TreeTools, DCAUtils, JLD2, PyPlot, Statistics, DelimitedFiles, PottsGauge
using AncestralSequenceReconstruction # core package
using JLD2 # used to load ArDCA models
using TreeTools # to handle p
using ArDCA, KitMSA
using Distributions

L = 50; q = 4; 

@load "../eq_sample_potts_t0.3_L50_d5.jld2"; 
h_diag = h; J_diag = J;

file_seqs = "../eq_sample_potts_t0.3_L50_d5.fa"



hs = [0 .* h, h, h]; Js = [J, J, 0 .*J] ; names = ["Only couplings", "Standard", "Profile"];
alphas = deepcopy(names); alpha = "Only_couplings"; 
 



eq_samples = [PhyloTools.read_rna(file_seqs,0.9) for _ in 1:3];

f1, f2 = compute_weighted_frequencies(Int8.(eq_samples[1]),q+1, 0.); c= reshape(f2 - f1*f1', q, L, q, L);
f1_0, f2_0 = compute_weighted_frequencies(Int8.(rand(1:q,L,5000)),q+1, 0.); c_0 = reshape(f2_0 - f1_0*f1_0', q, L, q, L);

close("all"); plt.hist(f1_0[:], histtype="step", label = "random"); plt.hist(f1[:], histtype = "step",label= "eq coloring"
    ); plt.xscale("log"); plt.yscale("log"); plt.title("One-point"); plt.legend(); savefig("../hist1.png")

close("all"); plt.hist(c_0[:], histtype="step", label = "random"); plt.hist(c[:], histtype = "step",label= "eq coloring"
    ); plt.xscale("log"); plt.yscale("log"); plt.title("C_ij"); plt.legend(); savefig("../hist2.png")


start_seq = [Int.(eq_samples[i][:,1]) for i in 1:length(alphas)];

ens_start = [PhyloTools.energy(start_seq[i], hs[i], Js[i]) for i in 1:length(alphas)];



tree_file = "../data_Anc/DBDtree_collapsed_noonlychild_midpointrooted_prunedsubtree301.nwk"
mus1 = [1., 2.,3., 4., 5.];
#tree_file = "../star_tree_301.nwk"
#mus1 = 0.33 .* [1., 2., 3., 4., 5.];
leaves_msas = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
hams_leaves =  zeros(length(alphas), length(mus1));
hams_intra_leaves = zeros(length(alphas), length(mus1));

num = 1;  
#generating data and reading it
#for alpha in names
    for n in 1:length(mus1) 
        @time res1 = run_evolution_ontree_amino(start_seq[num], tree_file, 
        hs[num], Js[num], mu = mus1[n], temp = 1., p = 0.5, q=q); 
        file_seqs = "../res_algos_coloring/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
        PhyloTools.leavestofasta_rna(file_seqs, res1)
    end
    #num += 1
#end

for n in 1:length(mus1)
    file_seqs = "../res_algos_coloring/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
    leaves_msas[num,n] = Int.(PhyloTools.read_rna(file_seqs,0.9))
    hams_leaves[num,n] = mean(ham_dist_nogap(start_seq[num], leaves_msas[num,n]))
    hams_intra_leaves[num,n] = pairwise_ham_dist(leaves_msas[num,n], n_seq = 300)
end

for n in 1:length(mus1)
    file_seqs = "../res_algos_coloring/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
    PhyloTools.convert_U_to_T_fasta(file_seqs)
end

hams_leaves


## doing felsenstein inference
res_ASR = zeros(Int, length(alphas), L, length(mus1));
pp = Matrix{Matrix{Float64}}(undef, length(alphas), length(mus1));
rec_msas = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
ens = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
hams = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
ens_ML =  zeros(length(alphas), length(mus1));
hams_ML =  zeros(length(alphas), length(mus1));


  
#inference with felsenstein

num = 1; #for alpha in names 
    for n in 1:length(mus1)
        file_seqs = "../res_algos_coloring/alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
        mu = infer_mu(tree_file, file_seqs)
        ### modify this
        res, p = PhyloTools.Felsenstein2rna(file_seqs, tree_file, file_seqs, mu)
        res_ASR[num,:,n] .= Int.(res)
        pp[num,n] = p
        rec_msas[num,n] = sampling_ANC(pp[num,n], n_seq = 500)
        hams[num,n] = ham_dist_nogap(start_seq[num], rec_msas[num,n])
        ens[num,n] = PhyloTools.energy(rec_msas[num,n],  hs[num],  Js[num]) .- ens_start[num]
        hams_ML[num,n] = ham_dist_nogap(start_seq[num], res_ASR[num,:,n])
        ens_ML[num,n] = PhyloTools.energy(res_ASR[num,:,n], hs[num],  Js[num]) - ens_start[num]
    end
    #num +=1
#end



#learning arDCA models on equilibrium data

ar_models = []
for i in 1:length(names)
    file_equil_seqs = "../eq_sample_potts_t0.3_L50_d5.fa"
    arnet,arvar=ardca(file_equil_seqs);
    push!(ar_models, AutoRegressiveModel(arnet))
end

#inference with arDCA

res_ASR_arDCA = zeros(Int, length(alphas), L, length(mus1));
rec_msas_arDCA = Matrix{Matrix{Int}}(undef, length(alphas), length(mus1));
ens_arDCA = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
hams_arDCA = Matrix{Array{Float64,1}}(undef, length(alphas), length(mus1));
ens_ML_arDCA =  zeros(length(alphas), length(mus1));
hams_ML_arDCA =  zeros(length(alphas), length(mus1));

num = 1; #for alpha in names
@time for n in 1:length(mus1)
        fasta_file = "../res_algos_coloring/ardca_T_alpha$(alpha)_potts_mu$(round(mus1[n], 
        digits = 2)).fa"
    
        strategy = ASRMethod(;joint = false, # (default) - joint reconstruction not functional yet
            ML = true, # (default)
            verbosity = 1, # the default is 0. 
            optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
            optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
            repetitions = 1, # (default) - for Bayesian reconstruction, multiple repetitions of the reconstruction process can be done to sample likely ancestors
            );
        opt_tree, reconstructed_sequences = infer_ancestral(
            tree_file, fasta_file, ar_models[num], strategy
            );
        println(typeof(reconstructed_sequences["NODE_1"]))
        println(reconstructed_sequences["NODE_1"])
        res_ASR_arDCA[num,:,n] = PhyloTools.string2vec_rna(reconstructed_sequences["NODE_1"]);
        node_list=["NODE_1"];
        
        
        #this commented part is for the bayesian arDCA method
        strategy_bayesian = ASRMethod(;
	joint = false, # (default) - joint reconstruction not functional yet
	ML = false, # (default)
	verbosity = 1, # the default is 0. 
	optimize_branch_length = false, # (default: false) - optimize the branch lengths of the tree using the evolutionary model
	optimize_branch_scale = false, # (default) - would optimize the branches while keeping their relative lengths fixed. Incompatible with the previous. 
	repetitions = 100,

)
        
    file_out = "../res_algos_coloring/ardca_inf/alpha$(alpha)_ardca_mu$(round(mus1[n],digits = 2))NODE_1.fa";
        infer_ancestral(
               tree_file, fasta_file, ar_models[num], strategy_bayesian;
               alignment_per_node=true, node_list = ["NODE_1"], outfasta = file_out);
    #PhyloTools.convert_T_to_U_fasta(file_out)
    println(n)
    end 
    #num += 1
#end

for n in 1:length(mus1)
    file_out = "../res_algos_coloring/ardca_inf/alpha$(alpha)_ardca_mu$(round(mus1[n],digits = 2))NODE_1_NODE_1.fa";
    PhyloTools.convert_T_to_U_fasta(file_out)
end
    

num = 1
#for alpha in alphas
    for n in 1:length(mus1)
        
        hams_ML_arDCA[num,n] = ham_dist_nogap(start_seq[num], res_ASR_arDCA[num,:,n])
        ens_ML_arDCA[num,n] = PhyloTools.energy(res_ASR_arDCA[num,:,n], hs[num], Js[num]) - ens_start[num] 
    
        rec_msas_arDCA[num,n] = PhyloTools.read_rna(
        "../res_algos_coloring/ardca_inf/ardca_U_alpha$(alpha)_ardca_mu$(round(mus1[n], digits = 2))NODE_1_NODE_1.fa", 0.9)
        
        hams_arDCA[num,n] = ham_dist_nogap(start_seq[num], Int.(rec_msas_arDCA[num,n]))
        ens_arDCA[num,n] = PhyloTools.energy(rec_msas_arDCA[num,n], hs[num], Js[num]) .- ens_start[num]
    end
    #num += 1
#end


close("all")
fig, axs = plt.subplots(figsize = (8,4))
num = 1; #for alpha in alphas
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(hams[num,n])./L for n in 1:length(mus1)] ,
    yerr = [std(hams[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, hams_ML[num,:]./L, marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(hams_arDCA[num,n])./L for n in 1:length(mus1)],
    yerr = [std(hams_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, hams_ML_arDCA[num,:]./L, marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end

fig.supxlabel("Leaves-root hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("Root-reconstruction \n hamming", fontsize = 20)
savefig("../single_alpha_ham_ASR_felse_vs_arDCA.png")


close("all")

fig, ax = plt.subplots(figsize=(8, 4))  # Single figure, single axis

# Plot Felse Bayes
ax.errorbar(hams_intra_leaves[num, :] ./ L, [mean(hams[num, n]) / L for n in 1:length(mus1)],
            yerr=[std(hams[num, n]) / L for n in 1:length(mus1)],
            color="blue", label="Felse bayes")

ax.scatter(hams_intra_leaves[num, :] ./ L, hams_ML[num, :] ./ L, marker="X", color="blue", s=100, label="Felse ML")

# Plot arDCA Bayes
ax.errorbar(hams_intra_leaves[num, :] ./ L, [mean(hams_arDCA[num, n]) / L for n in 1:length(mus1)],
            yerr=[std(hams_arDCA[num, n]) / L for n in 1:length(mus1)],
            color="red", label="arDCA bayes")

ax.scatter(hams_intra_leaves[num, :] ./ L, hams_ML_arDCA[num, :] ./ L, marker="X", s=100, color="red", label="arDCA ML")

# Labels and title
ax.set_title("Model = $(alpha)", fontsize=18)
ax.set_xlabel("Leaves pairwise hamming", fontsize=14)
ax.set_ylabel("Root-reconstruction hamming", fontsize=14)
ax.legend()

# Save the figure
savefig("../single_alpha_intra_ham_ASR_felse_vs_arDCA.png", dpi=300)


close("all")
fig, axs = plt.subplots(figsize = (8,4) )
num = 1; #for alpha in alphas
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(ens[num,n]) for n in 1:length(mus1)] ,
    yerr = [std(ens[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, ens_ML[num,:], marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_leaves[num,:] ./ L, [mean(ens_arDCA[num,n]) for n in 1:length(mus1)],
    yerr = [std(ens_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_leaves[num,:] ./ L, ens_ML_arDCA[num,:], marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end

fig.supxlabel("Leaves-root hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("E_ASR - E_wt", fontsize = 20)
savefig("../single_alpha_en_ASR_felse_vs_arDCA.png")


close("all")
fig, axs = plt.subplots(figsize = (8,4) )
num = 1; #for alpha in alphas
    axs.errorbar(hams_intra_leaves[num,:] ./ L, [mean(ens[num,n]) for n in 1:length(mus1)] ,
    yerr = [std(ens[num,n])./L for n in 1:length(mus1)], 
    color = "blue", label = "Felse bayes"); 
    axs.scatter(hams_intra_leaves[num,:] ./ L, ens_ML[num,:], marker = "X", color = "blue", s = 300, label = "Felse ML");
    
    axs.errorbar(hams_intra_leaves[num,:] ./ L, [mean(ens_arDCA[num,n]) for n in 1:length(mus1)],
    yerr = [std(ens_arDCA[num,n])./L for n in 1:length(mus1)], 
    color = "red", label = "arDCA bayes"); 
    axs.scatter(hams_intra_leaves[num,:] ./ L, ens_ML_arDCA[num,:] , marker = "X", s = 300, color = "red", label = "arDCA ML");
    axs.set_title("Model = $(alpha)", fontsize = 20)
    #num+=1
#end
fig.supxlabel("Leaves pairwise hamming", fontsize = 20)
axs.legend()
axs.set_ylabel("E_ASR - E_wt", fontsize = 20)
savefig("../single_alpha_intra_en_ASR_felse_vs_arDCA.png")



